# Random forest

### Imports

In [ ]:
#libraries
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#custom
import utils
import plots
import transformers as tran

LABEL = "1Y_default"
SEED = 42

%matplotlib inline  
%load_ext autoreload
%autoreload 2

random.seed(SEED)

### Load data

In [ ]:
#read from csvs
train_df = pd.read_csv("train_full.csv", na_filter = False)
test_df = pd.read_csv("test_full.csv", na_filter = False)

In [ ]:
#sanity checks
print("Train shape: {}".format(train_df.shape))
print("Test shape: {}".format(test_df.shape))
print("Train:")
utils.check_missing(train_df)
print("Test:")
utils.check_missing(test_df)

In [ ]:
#numerically encoded categoricals
to_categorical = ["Education", "EmploymentStatus", "Gender", "HomeOwnershipType", "LanguageCode", "MaritalStatus",
                 "NewCreditCustomer", "OccupationArea", "UseOfLoan", "VerificationType", "Country", "NrOfDependants", 
                  "NrOfDependants", "WorkExperience", "EmploymentDurationCurrentEmployer", "MonthlyPayment"]
for feature in to_categorical:
    train_df[feature] = train_df[feature].astype("category")
    test_df[feature] = test_df[feature].astype("category")

### First simple benchmark

In [ ]:
#model: RandomForest
rf_ben = RandomForestClassifier(n_estimators=100, random_state=SEED)  
print(rf_ben)
#prepare sets
ohe_rf = tran.OHE_transformer()
#train
X_train = ohe_rf.fit_transform(train_df.drop([LABEL], axis=1))
y_train = train_df[LABEL]
#test
X_test = ohe_rf.transform(test_df.drop([LABEL], axis=1))
y_test = test_df[LABEL]
#fit
rf_ben.fit(X_train, y_train)
#predict
train_preds_proba = rf_ben.predict_proba(X_train)[:,1]
test_preds_proba = rf_ben.predict_proba(X_test)[:,1]
#results
utils.print_results(train_preds_proba, y_train, test_preds_proba, y_test)
plots.plot_ROC_curve(rf_ben, X_test, y_test)
imps = plots.FeaturesImportanceTree(rf_ben, X_train.columns, figsize=(11,25), ret_idx=True)

## Hyper parametr tuning

In [ ]:
#rough grid
param_grid = {"n_estimators": np.arange(20, 320, 20),
              "max_features" : ["sqrt", "log2"]}

rf = RandomForestClassifier()
gs = GridSearchCV(rf, param_grid, scoring=utils.GINI_SCORER ,verbose=2)
gs.fit(X_train, y_train)

### Tuned model

In [ ]:
# rf = gs.best_estimator_
# print(rf)

rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=SEED,
            verbose=0, warm_start=False)

rf.fit(X_train, y_train)
#predict
train_preds_proba = rf.predict_proba(X_train)[:,1]
test_preds_proba = rf.predict_proba(X_test)[:,1]
#results
utils.print_results(train_preds_proba, y_train, test_preds_proba, y_test)
plots.plot_ROC_curve(rf, X_test, y_test)
plots.FeaturesImportanceTree(rf, X_train.columns, figsize=(11,25))

### Tune depth

In [ ]:
#depth "grid"
param_grid = {"max_depth" : [5, 10, 12, 15, 18, 20, None]}

rf_depth = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=SEED,
            verbose=0, warm_start=False)

gs = GridSearchCV(rf_depth, param_grid, scoring=utils.GINI_SCORER ,verbose=2)
gs.fit(X_train, y_train)

In [ ]:
rf_depth = gs.best_estimator_
print(rf_depth)

rf_depth.fit(X_train, y_train)
#predict
train_preds_proba = rf_depth.predict_proba(X_train)[:,1]
test_preds_proba = rf_depth.predict_proba(X_test)[:,1]
#results
utils.print_results(train_preds_proba, y_train, test_preds_proba, y_test)
plots.plot_ROC_curve(rf_depth, X_test, y_test)
plots.FeaturesImportanceTree(rf_depth, X_train.columns, figsize=(11,25))

### Model comparison

In [ ]:
models = [rf_ben, rf]
names = ["RF benchmark", "RF tuned"]

plots.plot_ROC_multiple(y_test, X_test, names, models)